# ... project description ...
https://github.com/comp-data/2021-2022/tree/main/docs/project

# What is done for Relational Database and Queries:

### Creating Database publications.db and Tables 
#### with CSV relational_publications.csv
#### with JSON relational_other_data.json

In [ ]:
pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=9a0343797d505aac41c14ec15384fd30a365619b6817327bca2790952a457e84
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
from sqlite3 import connect

with connect("publications6.db") as con:
    con.commit()  
    cursor = con.cursor()

### CSV...

'''
csv_file = open("./data/relational_publications.csv", encoding='utf-8')
contents = csv.reader(csv_file)
'''

In [ ]:
import pandas as pd
from pandas import read_csv, Series
import csv

csv_file = open("/content/drive/MyDrive/Colab Notebooks/dati/relational_publications.csv", encoding="utf-8")
contents = csv.reader(csv_file)

csv_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dati/relational_publications.csv", keep_default_na=False,
                        dtype={
                                    "id": "string",
                                    "title": "string",
                                    "type": "string",
                                    "publication_year": "string",
                                    "issue": "string",
                                    "volume": "string",
                                    "chapter": "string",
                                    "publication_venue": "string",
                                    "venue_type": "string",
                                    "publisher": "string",
                                    "event": "string"

                        },encoding="utf-8")

# Create a new column with internal identifiers for each publication
#csvfile_internal_id = []
#for idx, row in csv_file.iterrows():
#    csvfile_internal_id.append("publication-" + str(idx))
#csv_file.insert(0, "internalId", Series(csvfile_internal_id, dtype="string"))
csv_file

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,2020,1,1,,Quantitative Science Studies,journal,crossref:281,
1,doi:10.1007/s11192-019-03217-6,"Software Review: Coci, The Opencitations Index...",journal-article,2019,2,121,,Scientometrics,journal,crossref:297,
2,doi:10.1007/s11192-019-03311-9,Nine Million Book Items And Eleven Million Cit...,journal-article,2019,2,122,,Scientometrics,journal,crossref:297,
3,doi:10.1038/sdata.2016.18,The Fair Guiding Principles For Scientific Dat...,journal-article,2016,1,3,,Scientific Data,journal,crossref:297,
4,doi:10.1371/journal.pbio.3000385,The Nih Open Citation Collection: A Public Acc...,journal-article,2019,10,17,,Plos Biology,journal,crossref:340,
...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.3390/admsci10030069,Performance Analysis And Science Mapping Of In...,journal-article,2020,3,10,,Administrative Sciences,journal,crossref:1968,
496,doi:10.1186/s12888-020-02825-4,Mapping The Literature On Parents With Mental ...,journal-article,2020,1,20,,Bmc Psychiatry,journal,crossref:297,
497,doi:10.1080/00472778.2020.1776578,Evolution Of The Entrepreneurship And Innovati...,journal-article,2020,,,,Journal Of Small Business Management,journal,crossref:301,
498,doi:10.1007/s11301-020-00196-4,Intellectual Structure Of Management Innovatio...,journal-article,2020,3,71,,Management Review Quarterly,journal,crossref:297,


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
create_csvfile = '''
DROP TABLE IF EXISTS cvsfile;
CREATE TABLE IF NOT EXISTS csvfile
               (id, title, type, publication_year ,issue, volume, chapter, publication_venue, venue_type, publisher,event
              );
              '''

In [ ]:
cursor.executescript(create_csvfile)

In [ ]:

insert_records = "INSERT INTO csvfile (id, title, type, publication_year, issue, volume, chapter, publication_venue, venue_type, publisher, event) VALUES(?,?,?,?,?,?,?,?,?,?,?)"

cursor.executemany(insert_records, contents)

In [ ]:
cursor.executescript(create_csvfile)

#### ... finished with CSV ...

### Creating tables from JSON

In [ ]:
from json import load
import json
import pandas as pd

# importing authors from JSON

from collections import deque

with open("/content/drive/MyDrive/Colab Notebooks/dati/relational_other_data.json", "r", encoding="utf-8") as f:
    json_doc = load(f)
#print(type(json_doc))

# read just authors

authors = json_doc["authors"]

# create the dataframe from json

df_authors=pd.DataFrame(authors.items(),columns=['doi','author']).explode('author')

df_authors=pd.json_normalize(json.loads(df_authors.to_json(orient="records")))
df_authors.rename(columns={"author.family":"family_name","author.given":"given_name","author.orcid":"orc_id"}, inplace = True)

df_authors=pd.DataFrame(df_authors)

In [ ]:
cursor = con.cursor()
from pandasql import sqldf

def pysqldf(q):
    """this function eliminates the need to include locals/globals all the time"""
    return sqldf(q, globals())

#### Authors from JSON

In [ ]:
create_authors = '''
DROP TABLE IF EXISTS authors;

CREATE TABLE IF NOT EXISTS authors (
    orc_id      TEXT PRIMARY KEY,
    doi         TEXT,
    family_name TEXT,
    given_name  TEXT

);
'''

In [ ]:
cursor.executescript(create_authors)

In [ ]:
pysqldf('''select doi ,
         orc_id, family_name,
         given_name from df_authors;''').to_sql('authors',
                                               con, index=False,
                                               if_exists='replace')

#### Publisher from JSON

In [ ]:
### importing publisher JSON

# importing publisher JSON

# read just publishers

publisher = json_doc["publishers"]

# create the dataframe from json

df_publishers = pd.DataFrame(publisher.items(), columns=['crossref', 'name'])

df_publishers = pd.json_normalize(json.loads(df_publishers.to_json(orient="records")))
df_publishers.rename(columns={"name.id": "id", "name.name": "name"}, inplace=True)

In [ ]:
create_publishers = '''
DROP TABLE IF EXISTS publishers;

CREATE TABLE IF NOT EXISTS publishers (
    crossref    TEXT PRIMARY KEY,
    name TEXT
   );
'''

In [ ]:
cursor.executescript(create_publishers)

In [ ]:
pysqldf('''select crossref ,
           name from df_publishers;''').to_sql('publishers',
                                               con, index=False,
                                               if_exists='replace')

#### References from JSON

In [ ]:
references = json_doc["references"]

# create the dataframe from json

df_references = pd.DataFrame(references.items(), columns=['doi', 'refid']).explode('refid')

df_references = pd.json_normalize(json.loads(df_references.to_json(orient="records")))

#df_publishers.rename(columns={"name.id": "id", "name.name": "name"}, inplace=True)

In [ ]:
create_references = '''
DROP TABLE IF EXISTS references_json;

CREATE TABLE IF NOT EXISTS references_json (
    doi    TEXT,
    refid TEXT
   );
'''

In [ ]:
cursor.executescript(create_references)


In [ ]:
pysqldf('''select doi ,
           refid from df_references;''').to_sql('references_json',
                                               con, index=False,
                                               if_exists='replace')

#### Venues id from JSON

In [ ]:
venues_idjson = json_doc["venues_id"]

# create the dataframe from json

df_venuesjson=pd.DataFrame(venues_idjson.items(),columns=['doi','issn']).explode('issn')

df_venuesjson=pd.json_normalize(json.loads(df_venuesjson.to_json(orient="records")))
#df_venues.rename(columns={"author.family":"family_name","author.given":"given_name","author.orcid":"orc_id"}, inplace = True)

df_venuesjson=pd.DataFrame(df_venuesjson)

In [ ]:
create_venuesjson = '''
DROP TABLE IF EXISTS venuesid_json;

CREATE TABLE IF NOT EXISTS venuesid_json (
    doi    TEXT,
    issn TEXT PRIMARY KEY
   );
'''

In [ ]:
cursor.executescript(create_venuesjson)

In [ ]:
pysqldf('''select doi ,
           issn from df_venuesjson;''').to_sql('venuesid_json',
                                                con, index=False,
                                                if_exists='replace')

# ... finished with tables ....

#### Attempting creating methods that project requires.
#### They are not Class now... Only functionality

In [ ]:
#getProceedingsByEvent --> deve diventare il metodo della sua classe --> done

con.commit()
query= "SELECT * FROM csvfile WHERE publication_venue LIKE ? COLLATE NOCASE"
some_query_value = '%web%'
result= cursor.execute(query,[some_query_value]).fetchall()
df_getProceedingsByEvent = pd.DataFrame(result)

df_getProceedingsByEvent

,0,1,2,3,4,5,6,7,8,9,10
0,doi:10.1007/978-3-030-62466-8_28,The Opencitations Data Model,book-chapter,2020,,,1,Lecture Notes In Computer Science - The Semant...,book,crossref:297,
1,doi:10.1007/978-3-030-77385-4_37,Kgbench: A Collection Of Knowledge Graph Datas...,book-chapter,2021,,,1,The Semantic Web - Lecture Notes In Computer S...,book,crossref:297,


In [ ]:
#getPublicationsPublishedInYear --> by the way it should be programmed as method 
#in order to use in Class --> done 

query1= "SELECT * FROM csvfile WHERE publication_year= '2020'"
result1= cursor.execute(query1).fetchall()
df_getPublicationsPublishedInYear = pd.DataFrame(result1)

#df_getPublicationsPublishedInYear

In [ ]:
#getPublicationsByAuthorId --> deve diventare il metodo della sua classe --> done

query3= "SELECT* FROM authors WHERE orc_id='0000-0001-9857-1511'"

result=cursor.execute(query3).fetchall()
df_getPublicationsByAuthorId = pd.DataFrame(result)

df_getPublicationsByAuthorId

,0,1,2,3
0,doi:10.3989/arbor.2021.799007,0000-0001-9857-1511,Ortega,José Luis


In [ ]:
#getPublicationsByAuthorName --> deve diventare il metodo della sua classe --> done
query7= "SELECT* FROM authors WHERE given_name LIKE ? COLLATE NOCASE"
value= '%hara%'
result7= cursor.execute(query7,[value]).fetchall()
df_getPublicationsByAuthorName = pd.DataFrame(result7)

df_getPublicationsByAuthorName

,0,1,2,3
0,doi:10.1007/s11192-017-2481-0,0000-0002-3494-6801,Prabhakaran,Thara
1,doi:10.1016/j.jup.2018.06.003,0000-0002-3494-6801,Prabhakaran,Thara
2,doi:10.1080/21645515.2021.1910000,0000-0001-7630-8413,Harapan,Harapan


In [ ]:
#getVenuesByPublisherId --> deve diventare il metodo della sua classe --> done

query4= "SELECT* FROM publishers WHERE crossref='crossref:78'"
result4=cursor.execute(query4).fetchall()
df_getVenuesByPublisherId= pd.DataFrame(result4)

df_getVenuesByPublisherId

,0,1
0,crossref:78,Elsevier BV


In [ ]:
#getPublicationInVenue --> deve diventare il metodo della sua classe
query5= "SELECT* FROM venuesid_json WHERE issn='issn:0944-1344' "
result5=cursor.execute(query5).fetchall()
df_getPublicationInVenue= pd.DataFrame(result5)

df_getPublicationInVenue

,0,1
0,doi:10.1007/s11356-020-11947-x,issn:0944-1344
1,doi:10.1007/s11356-020-11643-w,issn:0944-1344
2,doi:10.1007/s11356-021-15939-3,issn:0944-1344
3,doi:10.1007/s11356-021-15787-1,issn:0944-1344
4,doi:10.1007/s11356-021-13094-3,issn:0944-1344


In [ ]:
#getJournalArticlesInIssue --> by the way it should be programmed as method 
#in order to use in Class --> done

query8= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
result8=cursor.execute(query8).fetchall()

for element in result8:
  query9= "SELECT* FROM csvfile WHERE type='journal-article' AND issue='9' AND volume='17' AND id=? "
  result9=cursor.execute(query9, element).fetchall()
df_getJournalArticlesInIssue= pd.DataFrame(result9)

df_getJournalArticlesInIssue

,0,1,2,3,4,5,6,7,8,9,10
0,doi:10.1080/21645515.2021.1910000,Research Trends In Rabies Vaccine In The Last ...,journal-article,2021,9,17,,Human Vaccines & Immunotherapeutics,journal,crossref:301,


In [ ]:
#getJournalArticlesInVolume --> by the way it should be programmed as method 
#in order to use in Class

query10= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
result10=cursor.execute(query10).fetchall()

for element in result10:
  query11= "SELECT* FROM csvfile WHERE type='journal-article' AND volume='17' AND id=? "
  result11=cursor.execute(query11, element).fetchall()
df_getJournalArticlesInVolume= pd.DataFrame(result11)

df_getJournalArticlesInVolume

,0,1,2,3,4,5,6,7,8,9,10
0,doi:10.1080/21645515.2021.1910000,Research Trends In Rabies Vaccine In The Last ...,journal-article,2021,9,17,,Human Vaccines & Immunotherapeutics,journal,crossref:301,


In [ ]:
#getJournalArticlesInJournal --> by the way it should be programmed as method 
#in order to use in Class

for element in result10:
  query12= "SELECT* FROM csvfile WHERE type='journal-article' AND id=?"
  result12=cursor.execute(query12, element).fetchall()
df_getJournalArticlesInJournal= pd.DataFrame(result12)

df_getJournalArticlesInJournal

,0,1,2,3,4,5,6,7,8,9,10
0,doi:10.1080/21645515.2021.1910000,Research Trends In Rabies Vaccine In The Last ...,journal-article,2021,9,17,,Human Vaccines & Immunotherapeutics,journal,crossref:301,


In [ ]:
#getPublicationAuthors --> deve diventare il metodo della sua classe
query6= "SELECT* FROM authors WHERE doi='doi:10.1080/21645515.2021.1910000'"
result6=cursor.execute(query6).fetchall()
df_getPublicationAuthors = pd.DataFrame(result6)

df_getPublicationAuthors

,0,1,2,3
0,doi:10.1080/21645515.2021.1910000,0000-0001-8793-273X,Ahmad,Tauseef
1,doi:10.1080/21645515.2021.1910000,0000-0003-0058-2031,Baig,Mukhtiar
2,doi:10.1080/21645515.2021.1910000,0000-0001-7630-8413,Harapan,Harapan


In [ ]:
##############################################################IGNORA###########################################################################
#getMostCitedPublication  --> by the way it should be programmed as method 
#in order to use in Class

query13="SELECT doi, COUNT(*) FROM references_json GROUP BY doi  "
result13=cursor.execute(query13).fetchall()


result13 = pd.DataFrame(result13)
lista=[]

for i  in range(len(result13)):
    lista.append(result13[1][i]) 

massimo= max(lista)

result13.columns = ['doi','citation']

getMostCitedPublication_df = result13.query("citation == @massimo")


In [ ]:
#getMostCitedPublication  --> by the way it should be programmed as method 
#in order to use in Class

query13="SELECT doi, count(*) FROM references_json GROUP BY doi ORDER BY COUNT (*) DESC LIMIT 5 " #non ho ancora ben capito come ma funziona
result13=cursor.execute(query13).fetchall()

df_getMostCitedPublication = pd.DataFrame(result13)

df_getMostCitedPublication


,0,1
0,doi:10.1162/qss_a_00023,5
1,doi:10.3390/ijerph18115985,5
2,doi:10.3989/arbor.2021.799007,5
3,doi:10.3145/thinkepi.2021.e15e04,4
4,doi:10.1007/978-3-030-62466-8_28,3


In [ ]:
#getMostCitedVenue --> by the way it should be programmed as method 
#in order to use in Class

query14= """SELECT *
FROM venuesid_json
INNER JOIN  (SELECT doi, count(*) FROM references_json GROUP BY doi ORDER BY COUNT (*) DESC LIMIT 5)
USING(doi)
"""    

result14=cursor.execute(query14).fetchall()


df_getMostCitedVenue = pd.DataFrame(result14)

df_getMostCitedVenue

,0,1,2
0,doi:10.1162/qss_a_00023,issn:2641-3337,5
1,doi:10.3390/ijerph18115985,issn:1660-4601,5
2,doi:10.3989/arbor.2021.799007,issn:1988-303X,5
3,doi:10.3989/arbor.2021.799007,issn:0210-1963,5
4,doi:10.3145/thinkepi.2021.e15e04,issn:2564-8837,4
5,doi:10.1007/978-3-030-62466-8_28,isbn:9783030624651,3
6,doi:10.1007/978-3-030-62466-8_28,isbn:9783030624668,3


#### some problems with implementing these:

##### getDistinctPublisherOfPublications
##### getDistinctPublisherOfPublications
##### getMostCitedPublication
####  getMostCitedVenue



 **E ora le classi...**

In [ ]:
class RelationalProcessor(object):
  def __init__(self,dbPath):
    self.dbPath= [] 

  def getdbPath(self):
    return str(RelationalProcessor.setdbPath(""))

  def setdbPath(self):
    dbPath= "publications6.db"
    return dbPath

In [ ]:
class RelationalQueryProcessor(RelationalProcessor): #provo a fare una sottoclasse 
        def __init__(self):
            self.self=self

        def getPublicationsPublishedInYear(self):
            query1= "SELECT * FROM csvfile WHERE publication_year= '2020'"
            result1= cursor.execute(query1).fetchall()
            df_getPublicationsPublishedInYear = pd.DataFrame(result1)
            return df_getPublicationsPublishedInYear

        def getPublicationsByAuthorId(self):
          query3= "SELECT* FROM authors WHERE orc_id='0000-0001-9857-1511'"
          result=cursor.execute(query3).fetchall()
          df_getPublicationsByAuthorId = pd.DataFrame(result)
          return df_getPublicationsByAuthorId

        def getVenuesByPublisherId(self):
          query4= "SELECT* FROM publishers WHERE crossref='crossref:78'"
          result4=cursor.execute(query4).fetchall()
          df_getVenuesByPublisherId= pd.DataFrame(result4)
          return df_getVenuesByPublisherId

        def getPublicationInVenue(self):
          query5= "SELECT* FROM venuesid_json WHERE issn='issn:0944-1344' "
          result5=cursor.execute(query5).fetchall()
          df_getPublicationInVenue= pd.DataFrame(result5)
          return df_getPublicationInVenue

        def getJournalArticlesInIssue(self):
          query8= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
          result8=cursor.execute(query8).fetchall()
          for element in result8:
            query9= "SELECT* FROM csvfile WHERE type='journal-article' AND issue='9' AND volume='17' AND id=? "
            result9=cursor.execute(query9, element).fetchall()
          df_getJournalArticlesInIssue= pd.DataFrame(result9)
          return df_getJournalArticlesInIssue

        def getJournalArticlesInVolume(self):
            query10= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
            result10=cursor.execute(query10).fetchall()
            for element in result10:
              query11= "SELECT* FROM csvfile WHERE type='journal-article' AND volume='17' AND id=? "
              result11=cursor.execute(query11, element).fetchall()
            df_getJournalArticlesInVolume= pd.DataFrame(result11)
            return df_getJournalArticlesInVolume

        def getJournalArticlesInJournal(self):
            query10= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
            result10=cursor.execute(query10).fetchall()
            for element in result10:
              query12= "SELECT* FROM csvfile WHERE type='journal-article' AND id=?"
              result12=cursor.execute(query12, element).fetchall()
            df_getJournalArticlesInJournal= pd.DataFrame(result12)
            return df_getJournalArticlesInJournal

        def getProceedingsByEvent (self):
            query= "SELECT * FROM csvfile WHERE publication_venue LIKE ? COLLATE NOCASE"
            some_query_value = '%web%'
            result= cursor.execute(query,[some_query_value]).fetchall()
            df_getProceedingsByEvent = pd.DataFrame(result)
            return df_getProceedingsByEvent

        def getPublicationsByAuthorName(self):
            query7= "SELECT* FROM authors WHERE given_name LIKE ? COLLATE NOCASE"
            value= '%hara%'
            result7= cursor.execute(query7,[value]).fetchall()
            df_getPublicationsByAuthorName = pd.DataFrame(result7)
            return df_getPublicationsByAuthorName

        def getPublicationAuthors (self): 
            query6= "SELECT* FROM authors WHERE doi='doi:10.1080/21645515.2021.1910000'"
            result6=cursor.execute(query6).fetchall()
            df_getPublicationAuthors = pd.DataFrame(result6)
            return df_getPublicationAuthors

        def getMostCitedPublication(self):
          query13="SELECT doi, count(*) FROM references_json GROUP BY doi ORDER BY COUNT (*) DESC LIMIT 5 " #non ho ancora ben capito come ma funziona
          result13=cursor.execute(query13).fetchall()
          df_getMostCitedPublication = pd.DataFrame(result13)
          return df_getMostCitedPublication

        def getMostCitedVenue(self):
          query14= """SELECT *
          FROM venuesid_json
          INNER JOIN  (SELECT doi, count(*) FROM references_json GROUP BY doi ORDER BY COUNT (*) DESC LIMIT 5)
          USING(doi)
          """    
          result14=cursor.execute(query14).fetchall()
          df_getMostCitedVenue = pd.DataFrame(result14)
          return df_getMostCitedVenue

#RelationalQueryProcessor.getPublicationsPublishedInYear('2020')
#RelationalQueryProcessor.getPublicationsByAuthorId('0000-0001-9857-1511')
#RelationalQueryProcessor.getVenuesByPublisherId('crossref:78')
#RelationalQueryProcessor.getPublicationInVenue('issn:0944-1344')
#RelationalQueryProcessor.getJournalArticlesInIssue('issn:2164-5515')
#RelationalQueryProcessor.getJournalArticlesInVolume('issn:2164-5515')
#RelationalQueryProcessor.getJournalArticlesInJournal('issn:2164-5515')
#RelationalQueryProcessor.getProceedingsByEvent('%web%')
#RelationalQueryProcessor.getPublicationsByAuthorName('%hara%')
#RelationalQueryProcessor.getPublicationAuthors('doi:10.1080/21645515.2021.1910000')
#RelationalQueryProcessor.getMostCitedPublication('')
#RelationalQueryProcessor.getMostCitedVenue('')



,0,1,2
0,doi:10.1162/qss_a_00023,issn:2641-3337,5
1,doi:10.3390/ijerph18115985,issn:1660-4601,5
2,doi:10.3989/arbor.2021.799007,issn:1988-303X,5
3,doi:10.3989/arbor.2021.799007,issn:0210-1963,5
4,doi:10.3145/thinkepi.2021.e15e04,issn:2564-8837,4
5,doi:10.1007/978-3-030-62466-8_28,isbn:9783030624651,3
6,doi:10.1007/978-3-030-62466-8_28,isbn:9783030624668,3
